In [ ]:
import pandas as pd
import numpy as np
import glob
import json
import requests as r
import math
pd.set_option('display.max_rows', 500)
import requests
from openpyxl import load_workbook

In [ ]:
def create_csvs_from_xls(filename):
    wb = load_workbook(filename=filename)
    problematic_sheets = {'policies'} # FIXME: remove after they are fixed and move to required_sheets
    required_sheets = {'resources', 'technologies', 'tags'} 
    for idx, sheet_name in enumerate(wb.sheetnames, start=1):
        sname = sheet_name.lower()
        if sname in problematic_sheets:
            print(f'NOTE: Data in {sheet_name} must be downloaded manually!!!!')
            print('^' * 50)
        if sname not in required_sheets:
            continue
        csv = f'{idx:-02}_{sheet_name.lower()}.csv'
        df = pd.read_excel(filename, sheet_name=sheet_name)
        df.columns = [c.strip().lower() for c in df.columns]
        df.to_csv(csv, index=False)
    return wb.sheetnames        

### GET ALL ORGANISTIONS

In [ ]:
def update_organisations():
    orglist = []
    for file in files:
        data = pd.read_csv(file)
        try:
            orgs = list(data[data['organisation'] == data['organisation']]['organisation'])
            for org in orgs:
                org = org.replace('"','').replace(',',';').split(';')
                for og in org:
                    og = og.strip()
                    if og != '':
                        orglist.append(og.strip())
        except:
            pass
    orglist = list(np.unique(orglist))
    orglist = [{"name":org} for org in orglist]
    with open(output_folder + 'organisations.json', 'w') as f:
        json.dump(orglist, f, indent=1, ensure_ascii=False)
        
    print(f'Updated {f.name}')

### Exploratory: LIST ALL MEA

In [ ]:
def get_unique_regional_coverage():
    #get unique reqgional coverage
    mea = []
    for file in files:
        df = pd.read_csv(file)
        try:
            df = df[df['geo_coverage'] == df['geo_coverage']]
            df['geo_coverage_type'] = df['geo_coverage'].apply(lambda x: x.split(':')[0].lower() if ':' in x else np.nan)
            df['geo_coverage'] = df['geo_coverage'].apply(lambda x: x.split(':')[1] if ':' in x else x)
            df = df[df['geo_coverage_type'] == df['geo_coverage_type']]
            df = df[['geo_coverage_type','geo_coverage']].to_dict('records')
            for d in df:
                if 'specific areas' in d['geo_coverage_type'] or d['geo_coverage_type'] == 'regional':
                    if ';' in d['geo_coverage']:
                        dd = d['geo_coverage'].split(';')
                        for gc in dd:
                            mea.append({'coverage':gc.strip(),'type': d['geo_coverage_type']})
                    else:
                        mea.append(mea.append({'coverage':d['geo_coverage'].strip(),'type':d['geo_coverage_type']}))
        except:
            print(file)
    mea = [i for i in mea if i]
    pd.DataFrame(mea).drop_duplicates(subset=['coverage'])
    new = pd.read_csv('./country_group.csv')
    new['source'] = 'new'
    old = pd.DataFrame(r.get("http://unep.localhost/api/public/groups").json())
    old['source_api'] = 'unep.tc'
    old = old[['name','source_api']]
    new['duplicates'] = new['name'].apply(lambda x: old.loc[old['name'].str.contains(x)].shape[0])

### GET ALL TAGS

In [ ]:
def update_tags():
    tags = pd.read_csv('./06_tags.csv')
    tags['category'] = tags['category'].apply(lambda x: x.split('_')[1])
    tags = tags[['category','tag']]
    tags = tags.groupby('category')['tag'].apply(lambda g: g.values.tolist()).to_dict()
    with open(output_folder + 'tags.json', 'w') as f:
        json.dump(tags, f, indent=1)
    print(f'Updated {f.name}')

### GET COUNTRY GROUPS

In [ ]:
def update_country_groups(data_file, country_group_names):
    # FIXME: Not sure how to generate the country_group.json
    # This code seems outdated, and we instead need country_group_countries
    # groups = pd.read_csv('./09.country_groups.csv')
    # groups['country'] = groups['country'].apply(lambda x: x.strip())
    # countryGroup = groups.groupby('group')['country'].apply(lambda g: g.values.tolist()).to_dict()
    # with open(output_folder + '/country_group.json', 'w') as file:
    #    file.write(json.dumps(countryGroup, indent=1))
    
    groups = {}
    for group_name in country_group_names:
        df = pd.read_excel(data_file, sheet_name=group_name, squeeze=True, header=None)
        groups[group_name] = df.apply(lambda x: x.strip()).to_list()
    
    with open(output_folder + 'country_group_countries.json', 'w') as f:
        json.dump(groups, f, indent=1, ensure_ascii=False)
    
    print(f'Updated {f.name}')

### GET ALL RESOURCES

In [ ]:
def get_currency(x):
    if "€" in x or "EUR" in x:
        return "EUR"
    if "$" in x or "USD" in x:
        return "USD"
    if "NOK" in x:
        return "NOK"
    if "CAD" in x:
        return "CAD"
    if "GBP" in x or "£":
        return "GBP"
    return None

In [ ]:
def get_value_currency(x):
    cur = [int(s) for s in x.split() if s.isdigit()]
    if len(cur) == 1:
        if "milli" in x:
            return cur[0] * 1000000
        if "billi" in x:
            return cur[0] * 1000000000
        return cur[0]
    return None

In [ ]:
def policy_date_transformer(date):
    if not date:
        return None
    if date == '18/18/2018':
        return '2018-01-01'    
    if '/' in date:
        day, month, year = date.split('/')
        return f'{year}-{month.zfill(2)}-{day.zfill(2)}'  
    if len(date) == 4:
        year = date
        return f'{year}-01-01'
    return date

def generate_theme(theme):
    converters = {
        'first_publication_date': policy_date_transformer, 
        'latest_amendment_date': policy_date_transformer
    }
    srcs = pd.read_csv(theme, converters=converters)
    taglist = pd.read_csv('./06_tags.csv')
    taglist = [t.strip() for t in list(taglist['tag'])]
    coverage_type = ['global',
                     'regional',
                     'national',
                     'transnational',
                     'sub-national',
                     'global with elements in specific areas']
    srcs = srcs.to_dict('records')
    resources = []
    false_tags = []
    for src in srcs:
        res = {}
        for s in src:
            res.update({s:src[s]})
            if type(src[s]) == float:
                if math.isnan(src[s]):
                    res.update({s: None})
            if s in ["publish_year","valid_from","valid_to"]:
                if res[s] is not None:
                    res.update({s: int(res[s])})
            if s == "url":
                if res[s] is not None:
                    res.update({"url": [v.strip() for v in res[s].splitlines(True)]})
            if s == "languages":
                languages = []
                if res[s] is not None:
                    lang = []
                    if ":" in res[s]:
                        lang = res[s].replace("http://","").replace("https://","").split(';')
                    if len(lang) > 0:
                        for ln in lang:
                            ln = ln.split(":")
                            if len(ln) > 1:
                                languages.append({"language":ln[0].strip(),"url":"https://{}".format(ln[1].strip())})
                if len(languages) > 0:
                    res.update({"resource_language_url": languages})
                else:
                    res.update({"resource_language_url": None})
                #del res[s]
            if type(res[s]) == str:
                v = res[s].replace('\n','').replace('"','').replace('‘','').replace('’','').replace('\xa0',' ')
                v = v.strip()
                res.update({s: v})
            ## Should we do data cleaning for value?
            if s == "value":
                if res[s] is not None:
                    res.update({"value_currency": get_currency(res[s])})
                    res.update({s: get_value_currency(res[s])})
                else:
                    res.update({"value_currency": None})
            if s in ["tags","organisation"]:
                if type(res[s]) == str:
                    vl = []
                    if res[s] is not None:
                        sep = [';' if ';' in src[s] else ':']
                        vl = res[s].split(sep[0])
                        vl = [k.replace('"','').strip() for k in vl]
                        if s == "tags":
                            nv = []
                            for tg in taglist:
                                for v in vl:
                                    if v == tg:
                                        nv.append(v)
                                    if v not in taglist:
                                        false_tags.append(v)
                            #if len(vl) != len(nv):
                            #    res.update({"error_tags":True})
                            #else:
                            #    res.update({"error_tags":False})
                            vl = nv
                    if len(vl) == 0:
                        vl = None
                    res.update({s: vl})
            if s == "geo_coverage":
                if res[s] is not None:
                    gt = res[s].split(":")
                    ct = gt[0].lower().strip()
                    if ct in coverage_type:
                        res.update({'geo_coverage_type': ct})
                    else:
                        res.update({'geo_coverage_type': None})
                    if len(gt) > 1:
                        gc = gt[1].split(';')
                        gc = [g.replace('.','').strip() for g in gc]
                        res.update({s: gc})
                    else:
                        res.update({s: None})
            if s == "attachments":
                if res[s] is not None:
                    res.update({s: res[s].split(' ')})
                else:
                    res.update({s: []})
            if s == "country":
                if res[s] is not None:
                    if "," in res[s]:
                        country = res[s].split(',')
                    elif ";" in res[s]:
                        country = res[s].split(';')
                    else:
                        country = [res[s]]
                    res.update({"country": country[0]})
                else:
                    res.update({"country": None})
        resources.append(res)
    results = pd.DataFrame(resources).to_dict('records')
    #df = df.fillna(dict(publish_year=999)).replace(dict(publish_year={999: None}))
    for res in results:
        for s in res:
            if type(res[s]) == float:
                if math.isnan(res[s]):
                    res.update({s: None})
                else:
                    res.update({s: int(res[s])})
    resources = pd.DataFrame(results)
    output_file = theme.split('_', 1)[1].replace('.csv','.json')
    with open(output_folder + output_file, 'w') as f:
        json.dump(results, f, indent=4 if 'resources' in theme else 1)
    print(f'Updated {f.name}')

In [ ]:
def update_countries(): 
    with open('countries.geojson', 'r') as f:
        country_data = json.load(f)
        
    countries = {}
    for feature in country_data['features']:
        props = feature['properties']
        name = props['name']
        if name.startswith('disputed'):
            continue
        if name not in countries:
            countries[name] = props['cd']
        elif 'Island' in countries[name] and 'Island' not in props['cd']:
            countries[name] = props['cd']
        else:
            print(f'{name} already mapped to {countries[name]}. Not adding {props}')
            
    countries = [{"name": val, "code": key} for key, val in countries.items()]
    countries = countries + [{'name': "All", "code": None}, {'name': "Other", "code": None}]
        
    with open(output_folder + 'countries.json', 'w') as f:
        json.dump(countries, f, indent=4, ensure_ascii=False)
    print(f'Updated {f.name}')

# Import/Update Projects

In [ ]:
actions = json.loads(requests.get('https://unep.tc.akvo.org/api/export/project-actions').content)

In [ ]:
for action in actions:
    if action['parent_id'] == 116:
        print(f"{action['code']}: '{action['name'].lower().split('(')[0].strip()}',")
        
geo_coverage_codes = {
    105885227: 'global',
    105885347: 'regional',
    105885443: 'transnational',
    105885568: 'national',
    105885616: 'sub-national',
    999999001: 'global with elements in specific areas',
    105994502: 'other',
}
geo_coverage_names = {val: key for key, val in geo_coverage_codes.items()}

In [ ]:
def transform_project_geo_coverage(project):
    countries = set(project['countries'])
    project_action_codes = set(project['action_codes'])
    
    if 'All' in countries:
        geo_coverage_type = 'global'
        project['countries'] = []
        
    elif len(countries) > 1 or geo_coverage_names['transnational'] in project_action_codes:
        geo_coverage_type = 'transnational'
        
    elif len(countries) == 1:
        if {geo_coverage_names['national'], geo_coverage_names['regional'], geo_coverage_names['other']}.intersection(project_action_codes):
            geo_coverage_type = 'national'
        elif geo_coverage_names['sub-national'] in project_action_codes:
            geo_coverage_type = 'sub-national'
        elif "Narrative Submission" in project['title']:
            geo_coverage_type = 'national'
        elif 'Other' in project['countries'] and geo_coverage_names['global'] in project_action_codes:
            geo_coverage_type = 'global'
            project['countries'] = []
        else:
            print("1 country project!!!")
            pprint(project['title'])
            pprint(project['countries'])
    else:
        geo_coverage_type = None
        #print("No countries??")
        #pprint(project['title'])
    project['geo_coverage_type'] = geo_coverage_type

In [ ]:
from pprint import pprint

def update_projects():
    project_data = json.loads(requests.get('https://unep.tc.akvo.org/api/export/projects').content)
    country_data = json.loads(requests.get('https://unep.tc.akvo.org/public/api/countries').content)

    TITLE_ACTION_CODE = 43374800
    SUMMARY_ACTION_CODE = 43374829
    
    old_names = {c['name']: c['code'] for c in country_data}
    with open(output_folder + 'countries.json', 'r') as f:
        new_names = {c['code']: c['name'] for c in json.load(f)}

    for project in project_data:
        for action_detail in project['action_details']:
            if action_detail['action_detail_code'] == TITLE_ACTION_CODE:
                title = action_detail['value']
                # Handle one ugly piece of data
                if title.startswith('1.\t'):
                    title = title[3:]
                project['title'] = title
            elif action_detail['action_detail_code'] == SUMMARY_ACTION_CODE:
                summary = action_detail['value']
                project['summary'] = summary
       
        # Replace "old" country names with new country names
        countries = []
        for name in project['countries']:
            if name not in old_names:
                print(project)
                print(f"{name} missing in old country list!")
                continue
            code = old_names[name]
            if code is not None:
                new_name = new_names.get(code)
                if new_name is None:
                    print(f"{code} ({name}) not found in new country name list")
                    continue
            else:
                new_name = name
            countries.append(new_name)
        project['countries'] = countries

        transform_project_geo_coverage(project)

    with open('../backend/dev/resources/files/projects.json', 'w') as f:
        json.dump(project_data, f, indent=2, ensure_ascii=False)
    print(f'Updated {f.name}')

In [ ]:
data_file = 'flat-data-structure.xlsx'
output_folder = "../backend/dev/resources/files/"
sheetnames = create_csvs_from_xls(data_file)
files = glob.glob("./0*.csv")
print(files)

update_countries()  # NOTE: This is slightly slower, because we open the huge geojson
update_organisations()
update_tags()
country_group_names = sheetnames[sheetnames.index('projects')+1:]
update_country_groups(data_file, country_group_names)
generate_theme("05_technologies.csv")
generate_theme("04_policies.csv")
generate_theme("02_resources.csv")
# NOTE: Always call update_projects after update_countries
update_projects()

#### NOTES
- Do we need to analyze value and it's currency?
- No Country Table
- Country also has **global with elements in specific areas**
- Some of the country has different separator
- Some of the geo_coverage has different separator
- Some of the tags has different separator
- languages separator is using colon while the url is also using colon